In [27]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import seaborn as sns
from pandasgui import show
import socket



Transactions CM 2021 & 2023 

In [54]:
transac_cm_2021_path = os.path.join('..', 'interm_data', 'yearly_data', 'Transacciones', 'transacciones_cm_2021.csv')
transac_cm_2021_df = pd.read_csv(transac_cm_2021_path)
transac_cm_2021_df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'], inplace=True)
print(f"Dimensions of transac_cm_2021_df: {transac_cm_2021_df.shape}")
transac_cm_2021_df.head()

Dimensions of transac_cm_2021_df: (1071, 45)


,Nro Licitación Pública,Id Convenio Marco,Convenio Marco,CodigoOC,NombreOC,Fecha Envío OC,EstadoOC,Proviene de Gran Compra,idGranCompra,Especificación del Comprador,...,Nombre Proveedor Sucursal,Nombre Empresa,Comuna del Proveedor,Región del Proveedor,Observaciones,Forma de Pago,Orgcode_Comprador,Entcode_Comprador,Orgcode_Proveedor,Entcode_Proveedor
0,2239-5-LR21,5800296.0,CM Adquisición de Vehículos SUV y Camionetas (...,2564-270-CM21,Orden de Compra: 2564-270-CM21,2021-10-20,Enviada a Proveedor,No,NaN,(1840093) CAMIONETAS MAXUS T60 T60 DX 4X4 MAC...,...,Mediterraneo Automotores SA,Mediterraneo Automotores S.A.,NaN,NaN,Orden de Compra codigo: 2564-270-CM21 dirigida...,30 días contra la recepción conforme de la fac...,3555.0,92414.0,824156,27979
1,2239-5-LR21,5800296.0,CM Adquisición de Vehículos SUV y Camionetas (...,4360-75-CM21,Camioneta maxus d/c 4x4,2021-12-24,Enviada a Proveedor,No,NaN,(1840095) CAMIONETAS MAXUS T60 T60 DX 4X4 MAC...,...,Mediterraneo Automotores SA,Mediterraneo Automotores S.A.,NaN,NaN,Orden de Compra codigo: 4360-75-CM21 dirigida ...,30 días contra la recepción conforme de la fac...,5277.0,135561.0,824156,27979
2,2239-5-LR21,5800296.0,CM Adquisición de Vehículos SUV y Camionetas (...,3140-819-CM21,"Orden de Compra: 3140-819-CM21, ADQUISICIÓN DE...",2021-12-21,Enviada a Proveedor,No,NaN,(1848052) CAMIONETA NISSAN NAVARA DC SE 2.3D M...,...,Salinas y Fabres S.A.,SALINAS Y FABRES SOCIEDAD ANONIMA,NaN,NaN,Orden de Compra codigo: 3140-819-CM21 dirigida...,30 días contra la recepción conforme de la fac...,4061.0,113735.0,26569,26778
3,2239-5-LR21,5800296.0,CM Adquisición de Vehículos SUV y Camionetas (...,1084170-110-CM21,Camioneta para Uso del SLEP,2021-12-28,Recepción Conforme,No,NaN,(1840060) CAMIONETAS MAHINDRA PIK UP PICK UP R...,...,BRUNO FRITSCH S.A.,BRUNO FRITSCH S.A.,NaN,NaN,1.\tAceptar la OC y facturar después de entreg...,30 días contra la recepción conforme de la fac...,1084170.0,1675231.0,24507,24823
4,2239-5-LR21,5800296.0,CM Adquisición de Vehículos SUV y Camionetas (...,4236-722-CM21,Orden de Compra: 4236-722-CM21,2021-12-29,Aceptada,No,NaN,(1848008) CAMIONETA TOYOTA HILUX 4X4 2.4 MT DX...,...,Maritano y Ebensperger Ltda.,Maritano y Ebensperger Ltda.,NaN,NaN,Orden de Compra codigo: 4236-722-CM21 dirigida...,30 días contra la recepción conforme de la fac...,5154.0,127036.0,62896,61164


In [55]:
transac_cm_2023_path = os.path.join('..', 'interm_data',  'yearly_data', 'Transacciones', 'transacciones_cm_2023.csv')
transac_cm_2023_df = pd.read_csv(transac_cm_2023_path)
transac_cm_2023_df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'], inplace=True)
print(f"Dimensions of transac_cm_2023_df: {transac_cm_2023_df.shape}")
transac_cm_2023_df.head()

Dimensions of transac_cm_2023_df: (366, 45)


,Nro Licitación Pública,Id Convenio Marco,Convenio Marco,CodigoOC,NombreOC,Fecha Envío OC,EstadoOC,Proviene de Gran Compra,idGranCompra,Especificación del Comprador,...,Nombre Proveedor Sucursal,Nombre Empresa,Comuna del Proveedor,Región del Proveedor,Observaciones,Forma de Pago,Orgcode_Comprador,Entcode_Comprador,Orgcode_Proveedor,Entcode_Proveedor
0,2239-8-LR23,5802342.0,Convenio Marco de Adquisición de Vehículos,4062-187-CM24,Orden de Compra: 4062-187-CM24,2024-03-12,Recepcion Conforme,No,NaN,(2042525) CAMIONETA MAXUS T60 E6 GL 4X4 MT 202...,...,SERVIMAQ SPA,AUTOMOTRIZ SERVIMAQ SPA,NaN,NaN,Orden de Compra codigo: 4062-187-CM24 dirigida...,30 dias contra la recepcion conforme de la fac...,4982,120001,100329,97856
1,2239-8-LR23,5802342.0,Convenio Marco de Adquisición de Vehículos,3577-41-CM24,Orden de Compra: 3577-41-CM24,2024-03-14,Aceptada,No,NaN,(2041399) CAMIONETA TOYOTA HILUX DX 2.4 4X4 MT...,...,BRUNO FRITSCH S.A.,BRUNO FRITSCH S.A.,NaN,NaN,Orden de Compra codigo: 3577-41-CM24 dirigida ...,30 dias contra la recepcion conforme de la fac...,4497,115285,24507,24823
2,2239-8-LR23,5802342.0,Convenio Marco de Adquisición de Vehículos,3030-177-CM24,CAMIONETA JMC PARA DEPTO. DE SALUD,2024-03-14,Aceptada,No,NaN,(2041534) CAMIONETA JMC VIGUS NEW WORK 4X4 MT ...,...,SALAZAR ISRAEL,AUTOMOTRIZ CORDILLERA S.A.,NaN,NaN,CAMIONETA JMC VIGUS NEW WORK 4X4 MT 2024 PARA ...,30 dias contra la recepcion conforme de la fac...,3978,113252,32639,32414
3,2239-8-LR23,5802342.0,Convenio Marco de Adquisición de Vehículos,2381-308-CM24,ADQUISICIÓN DE 01 CAMIONETA TODO TERRENO PARA ...,2024-03-18,Aceptada,No,NaN,(2041392) CAMIONETA TOYOTA HILUX DX 2.4 4X4 MT...,...,BRUNO FRITSCH S.A.,BRUNO FRITSCH S.A.,NaN,NaN,ADQUISICIÓN DE 01 CAMIONETA TODO TERRENO PARA ...,30 dias contra la recepcion conforme de la fac...,3373,86872,24507,24823
4,2239-8-LR23,5802342.0,Convenio Marco de Adquisición de Vehículos,316-212-CM24,Orden de Compra: 316-212-CM24,2024-03-20,Aceptada,No,NaN,(2041566) CAMIONETA JMC VIGUS NEW PLUS 4X4 MT ...,...,SALAZAR ISRAEL,AUTOMOTRIZ CORDILLERA S.A.,NaN,NaN,Orden de Compra codigo: 316-212-CM24 dirigida ...,30 dias contra la recepcion conforme de la fac...,1313,7233,32639,32414


Join the purchases from 2021 and 2023

In [56]:
# join bot dataframes 

# Add the year column to each dataframe
transac_cm_2021_df['year'] = 2021
transac_cm_2023_df['year'] = 2023

# Concatenate the dataframes vertically
transac = pd.concat([transac_cm_2021_df, transac_cm_2023_df], axis=0)

# Reset the index if needed
transac_df = transac.reset_index(drop=True)
print(transac_df.columns)
print(f"Dimensions of transac_df: {transac_df.shape}")
transac_df.head(2)
#show(transac_df)

Index(['Nro Licitación Pública', 'Id Convenio Marco', 'Convenio Marco',
       'CodigoOC', 'NombreOC', 'Fecha Envío OC', 'EstadoOC',
       'Proviene de Gran Compra', 'idGranCompra',
       'Especificación del Comprador', 'IDProductoCM', 'Producto',
       'Nombre Producto ONU', 'Tipo de Producto', 'Marca', 'Modelo',
       'Precio Unitario', 'Cantidad', 'TotaLínea(Neto)', 'Moneda',
       'Monto Total OC Neto', 'Descuento Global OC', 'Cargos Adicionales OC',
       'Subtotal OC', 'Impuestos', 'Monto Total OC', 'Rut Unidad de Compra',
       'Unidad de Compra', 'Razón Social Comprador', 'Dirección Unidad Compra',
       'Comuna Unidad Compra', 'Región Unidad de Compra', 'Institución',
       'Sector', 'Rut Proveedor', 'Nombre Proveedor Sucursal',
       'Nombre Empresa', 'Comuna del Proveedor', 'Región del Proveedor',
       'Observaciones', 'Forma de Pago', 'Orgcode_Comprador',
       'Entcode_Comprador', 'Orgcode_Proveedor', 'Entcode_Proveedor', 'year'],
      dtype='object')
Dimensi

,Nro Licitación Pública,Id Convenio Marco,Convenio Marco,CodigoOC,NombreOC,Fecha Envío OC,EstadoOC,Proviene de Gran Compra,idGranCompra,Especificación del Comprador,...,Nombre Empresa,Comuna del Proveedor,Región del Proveedor,Observaciones,Forma de Pago,Orgcode_Comprador,Entcode_Comprador,Orgcode_Proveedor,Entcode_Proveedor,year
0,2239-5-LR21,5800296.0,CM Adquisición de Vehículos SUV y Camionetas (...,2564-270-CM21,Orden de Compra: 2564-270-CM21,2021-10-20,Enviada a Proveedor,No,NaN,(1840093) CAMIONETAS MAXUS T60 T60 DX 4X4 MAC...,...,Mediterraneo Automotores S.A.,NaN,NaN,Orden de Compra codigo: 2564-270-CM21 dirigida...,30 días contra la recepción conforme de la fac...,3555.0,92414.0,824156,27979,2021
1,2239-5-LR21,5800296.0,CM Adquisición de Vehículos SUV y Camionetas (...,4360-75-CM21,Camioneta maxus d/c 4x4,2021-12-24,Enviada a Proveedor,No,NaN,(1840095) CAMIONETAS MAXUS T60 T60 DX 4X4 MAC...,...,Mediterraneo Automotores S.A.,NaN,NaN,Orden de Compra codigo: 4360-75-CM21 dirigida ...,30 días contra la recepción conforme de la fac...,5277.0,135561.0,824156,27979,2021


Pair the variables of the model with the variables of the data     
        
    'IDProductoCM' -> j 
    To create product characteristics: 'Tipo de Producto', 'Marca', 'Nombre Producto ONU'
    'Precio Unitario'-> net not including taxes, just changes the scale of the price parameter 
    'Rut Unidad de Compra' -> i 
    To create the groups (k):  'Región Unidad de Compra', 'Sector'
     'year' -> t 

     'Modelo'-> to do the match with the product characteristics. 
        
Variables not directly paired but which could be useful: 
    'Nro Licitación Pública', 'Id Convenio Marco', 'Convenio Marco', 'CodigoOC', 'Fecha Envío OC', 'Cantidad', 'Rut Proveedor', 
    'Nombre Proveedor Sucursal', 'Orgcode_Comprador', 'Entcode_Comprador', 


In [64]:
# Variables to keep - directly paired with model
model_vars = ['IDProductoCM', 'Modelo',  'Tipo de Producto', 'Marca', 'Nombre Producto ONU',
              'Precio Unitario', 'Rut Unidad de Compra', 'Región Unidad de Compra',
              'Sector', 'year']

# Additional useful variables
extra_vars = ['Nro Licitación Pública', 'Id Convenio Marco', 'Convenio Marco',
              'CodigoOC', 'Fecha Envío OC', 'Cantidad', 'Rut Proveedor',
              'Nombre Proveedor Sucursal', 'Orgcode_Comprador', 'Entcode_Comprador']

# Create new dataframe with only the specified columns
transac_df = transac_df[ model_vars + extra_vars]



print(f"Dimensions of transac_df: {transac_df.shape}")
transac_df.head(2)
show(transac_df)


PandasGUI INFO — pandasgui.gui — Opening PandasGUI


Dimensions of transac_df: (1437, 20)


Answer the following questions 
- are firms selling multiple products? Yes


In [58]:
# Group by supplier and count unique products
supplier_products = transac_df.groupby('Nombre Proveedor Sucursal')['IDProductoCM'].nunique().sort_values(ascending=False)
print(supplier_products)

n_unique_products = transac_df['IDProductoCM'].nunique()
print(f"Total number of unique products: {n_unique_products}")

Nombre Proveedor Sucursal
SALAZAR ISRAEL                                   153
Salinas y Fabres S.A.                            111
Automotores Gildemeister CAMIONES                 57
BRUNO FRITSCH S.A.                                50
COMERCIAL ANFRUNS Y COMPAÑIA SPA                  46
Derco SPA                                         33
Maritano y Ebensperger Ltda.                      29
Kovacs SpA                                        21
Mediterraneo Automotores SA                       21
Comercial Motores de los Andes SpA                15
SERVIMAQ SPA                                      12
Automotora Comercial Costabal y Echenique S.A      7
DERCOMAQ SPA                                       6
Automotriz Tecnosur Limitada                       5
POMPEYO CARRASCO Y CIA. LTDA.                      5
PIAMONTE S.A.                                      4
PORSCHE CHILE SPA                                  3
KIA CHILE SPA                                      3
AUTOMOTRIZ PORTILLO 

Merge product characteristics

In [59]:
# Assuming transac_df is already defined and loaded with data
unique_modelo_values = transac_df['Modelo'].unique()
print(unique_modelo_values)

['T60 T60 DX 4X4' 'NAVARA DC SE 2.3D MT 4WD'
 'PIK UP PICK UP REFRESH DC 4X2 MT FULL' 'HILUX 4X4 2.4 MT DX'
 'X-TRAIL SENSE 2ROW 4X4 G2' 'X-TRAIL SENSE 2ROW 4X2 G2'
 'L-200 KATANA CRM 4X4' 'NAVARA DC SE 2.3D MT' 'L-200 KATANA CRT 4X4'
 'WINGLE 5 G 2.2 4X2 ELITE' 'T60 DX 4X4' 'VENUE QX 1.6L MT PLUS'
 'T60 DX 4X2' 'HILUX 4X2 2.4 MT DX' 'VENUE QX 1.6 MT PLUS'
 'WINGLE 7 GL 4X2 ELITE' 'VIGUS DOBLE CABINA 4X2 DIESEL'
 'VIGUS WORK LX 4X4 DIESEL G4' 'SAVEIRO DOBLE CABINA'
 'OROCH OROCH  ZEN 1,6L MT 4X2' 'T60 2.8L GL 4X2' 'T60 2.8L DX 4X4'
 'X-TRAIL 2.5L SENSE AUT' 'VENUE QX 1.6 MT VALUE' '700 1.4L BIGHORN'
 'NAVARA 2.3D DC XE' 'NAVARA 2.3D DC XE 4X4' '700 1.4L SLT CD'
 'QASHQAI 1.3L ADVANCE MT 2WD' 'KICKS 1.6L ADVANCE' 'VIGUS PLUS 2.4L 4X4'
 '700 1.4L SLT' 'TIGGO 7 PRO 1.5L GLS AUT' 'KICKS 1.6L SENSE'
 'TIGGO 3 1.5L GLS' 'VIGUS WORK 2.5L 4X4' 'TIGGO 8 1.5L GLS'
 'L200 KATANA CRT 2.4L 4X4' 'HILUX 2.4L DX CD 4X2' 'HILUX 2.4L DX CD 4X4'
 'L200 KATANA CR 2.4L 4X2' 'NAVARA 2.3D DC SE 4X4'
 'X-TRAI

In [67]:
# import the product characteristics
prod_char_path = os.path.join('..', 'car_data',   'scraped_data.csv')
prod_char_df = pd.read_csv(prod_char_path)
prod_char_df.rename(columns={'MODEL': 'Modelo'}, inplace=True)

prod_char_df['matching var'] = prod_char_df['MARCA'] + ' ' + prod_char_df['Model Name'] + ' ' + prod_char_df['Drive']


prod_char_df.head()





,Modelo,MARCA,Model Name,Engine,Drive,Transmission,Year,Version URL,Motor - Combustible,Motor - Cilindrada,...,Motor eléctrico - Voltaje,Confort - Palanca de cambios,Transmisión y chasis - Neumáticos delanteros,Transmisión y chasis - Neumáticos traseros,Motor - Potencia total sistema híbrido,Seguridad - Cabeceras delanteras activas,Comunicación y entretenimiento - Preinstalación teléfono,Comunicación y entretenimiento - Disco rígido,Confort - Puertas,matching var
0,CHANGAN HUNTER COMFORT 2.0T 4X2 MT 2024,CHANGAN,HUNTER COMFORT,2.0T,4X2,MT,2024,https://www.autocosmos.cl/catalogo/2024/changa...,diesel,1910 cc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHANGAN HUNTER COMFORT 4X2
1,CHANGAN HUNTER COMFORT 2.0T 4X4 MT 2024,CHANGAN,HUNTER COMFORT,2.0T,4X4,MT,2024,https://www.autocosmos.cl/catalogo/2024/changa...,diesel,1910 cc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHANGAN HUNTER COMFORT 4X4
2,CHANGAN HUNTER ELITE SPORT 2.0T 4X4 MT 2024,CHANGAN,HUNTER ELITE SPORT,2.0T,4X4,MT,2024,https://www.autocosmos.cl/catalogo/2024/changa...,diesel,1910 cc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHANGAN HUNTER ELITE SPORT 4X4
3,CHANGAN HUNTER LUXURY 2.0T 4X2 MT 2024,CHANGAN,HUNTER LUXURY,2.0T,4X2,MT,2024,https://www.autocosmos.cl/catalogo/2024/changa...,diesel,1910 cc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHANGAN HUNTER LUXURY 4X2
4,CHANGAN MD201 MT 2024,CHANGAN,MD201,NaN,NaN,MT,2024,https://www.autocosmos.cl/catalogo/2024/changa...,bencina,1243 cc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
transac_df['matching var'] = transac_df['Marca'] + ' ' + transac_df['Modelo']

# Get number of rows before merge
n_before = len(transac_df)

# Merge dataframes
transac_df = transac_df.merge(prod_char_df, on='matching var', how='left')

# Get number of rows after merge
n_after = len(transac_df)

# Get number of matched rows by checking any column from prod_char_df
# Replace 'some_column_from_prod_char_df' with an actual column name from prod_char_df
n_matched = transac_df['Version URL'].notna().sum()

print(f"Observations before merge: {n_before}")
print(f"Observations after merge: {n_after}")
print(f"Number of matched observations: {n_matched}")
print(f"Number of unmatched observations: {n_after - n_matched}")
print(f"Percentage of matched observations: {(n_matched/n_after*100):.2f}%")

KeyError: 'Modelo'